In [2]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [3]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 765.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


每10个epoch save一遍model, save output, 用val generate一遍output, 最后跑完再用test data generate一遍（这个我觉得可以等我们觉得model效果ok了以后再跑，防止他memorize test data）--》 用作对比for qualitative and quantitative data

再对d loss和g loss都画个图

解决一下生成出来时黑白的问题

train func也得改改




In [17]:
main_dir = '/content/drive/MyDrive/Colab Notebooks/Baseline Model'

photo_train_dir = main_dir + '/photos_train_student_removeBG'
photo_test_dir = main_dir + '/photos_test_student_removeBG'
photo_val_dir = main_dir + '/photos_val_student_removeBG'

sketch_train_dir = main_dir + '/sketch_train_studentSaved'
sketch_test_dir = main_dir + '/sketch_test_studentSaved'
sketch_val_dir = main_dir + '/sketch_val_studentSaved'

In [31]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ImageFolderDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(('jpg', 'png', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

In [1]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load datasets
photo_train_dataset = ImageFolderDataset(photo_train_dir, transform=transform)
photo_test_dataset = ImageFolderDataset(photo_test_dir, transform=transform)
photo_val_dataset = ImageFolderDataset(photo_val_dir, transform=transform)
sketch_train_dataset = ImageFolderDataset(sketch_train_dir, transform=transform)
sketch_test_dataset = ImageFolderDataset(sketch_test_dir, transform=transform)
sketch_val_dataset = ImageFolderDataset(sketch_val_dir, transform=transform)

NameError: name 'transforms' is not defined

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image

# Define the Generator
class UNetGenerator(nn.Module):
    def __init__(self, in_channels, out_channels, features=64):
        super(UNetGenerator, self).__init__()
        self.encoder = nn.Sequential(
            self.contract_block(in_channels, features, False),
            self.contract_block(features, features * 2),
            self.contract_block(features * 2, features * 4),
            self.contract_block(features * 4, features * 8),
        )
        self.decoder = nn.Sequential(
            self.expand_block(features * 8, features * 4),
            self.expand_block(features * 4, features * 2),
            self.expand_block(features * 2, features),
            nn.ConvTranspose2d(features, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh(),
        )

    def __call__(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def contract_block(self, in_channels, out_channels, use_bn=True):
        block = [nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)]
        if use_bn:
            block.append(nn.BatchNorm2d(out_channels))
        block.append(nn.ReLU(inplace=True))
        return nn.Sequential(*block)

    def expand_block(self, in_channels, out_channels):
        block = [
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        ]
        return nn.Sequential(*block)

# Define the Discriminator
class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels, features=64):
        super(PatchDiscriminator, self).__init__()
        self.model = nn.Sequential(
            self.discriminator_block(in_channels * 2, features, False),
            self.discriminator_block(features, features * 2),
            self.discriminator_block(features * 2, features * 4),
            nn.Conv2d(features * 4, 1, kernel_size=3, padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x, y):
        return self.model(torch.cat([x, y], dim=1))

    def discriminator_block(self, in_channels, out_channels, use_bn=True):
        block = [nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)]
        if use_bn:
            block.append(nn.BatchNorm2d(out_channels))
        block.append(nn.LeakyReLU(0.2, inplace=True))
        return nn.Sequential(*block)

# Training Loop
def train_gan(generator, discriminator, photo_dataloader, sketch_dataloader, num_epochs=200, lambda_l1=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator = generator.to(device)
    discriminator = discriminator.to(device)

    criterion_gan = nn.BCELoss()
    criterion_l1 = nn.L1Loss()

    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    output_dir = main_dir + '/output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for epoch in range(num_epochs):
        for i, (photo_batch, sketch_batch) in enumerate(zip(photo_dataloader, sketch_dataloader)):
            photo_batch = photo_batch.to(device)
            sketch_batch = sketch_batch.to(device)

            # Train Discriminator
            optimizer_d.zero_grad()
            fake_sketch = generator(photo_batch)
            real_validity = discriminator(photo_batch, sketch_batch)
            fake_validity = discriminator(photo_batch, fake_sketch.detach())

            real_loss = criterion_gan(real_validity, torch.ones_like(real_validity, device=device))
            fake_loss = criterion_gan(fake_validity, torch.zeros_like(fake_validity, device=device))
            d_loss = (real_loss + fake_loss)
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            fake_validity = discriminator(photo_batch, fake_sketch)
            gan_loss = criterion_gan(fake_validity, torch.ones_like(fake_validity, device=device))
            l1_loss = criterion_l1(fake_sketch, sketch_batch)
            g_loss = gan_loss + lambda_l1 * l1_loss
            g_loss.backward()
            optimizer_g.step()

            if i % 100 == 0:
                print(f"Epoch [{epoch}/{num_epochs}] Batch {i}/{len(photo_dataloader)} \
                      Loss D: {d_loss.item()}, loss G: {g_loss.item()}")

        # Save some generated images for visualization

        save_image(fake_sketch[:25], f'{output_dir}/{epoch}_generated.png', nrow=5, normalize=True)

# Load datasets
photo_train_dataset = ImageFolderDataset(photo_train_dir, transform=transform)
photo_test_dataset = ImageFolderDataset(photo_test_dir, transform=transform)
photo_val_dataset = ImageFolderDataset(photo_val_dir, transform=transform)
sketch_train_dataset = ImageFolderDataset(sketch_train_dir, transform=transform)
sketch_test_dataset = ImageFolderDataset(sketch_test_dir, transform=transform)
sketch_val_dataset = ImageFolderDataset(sketch_val_dir, transform=transform)

photo_train_loader = DataLoader(photo_train_dataset, batch_size=32, shuffle=True)
sketch_train_loader = DataLoader(sketch_train_dataset, batch_size=32, shuffle=True)

# Initialize models
generator = UNetGenerator(in_channels=3, out_channels=3)
discriminator = PatchDiscriminator(in_channels=3)

# Train the model
train_gan(generator, discriminator, photo_train_loader, sketch_train_loader, num_epochs=200)

Epoch [0/200] Batch 0/6                       Loss D: 0.7071510553359985, loss G: 86.29266357421875
Epoch [1/200] Batch 0/6                       Loss D: 0.5914514660835266, loss G: 58.717079162597656
Epoch [2/200] Batch 0/6                       Loss D: 0.4685845375061035, loss G: 48.92205810546875
Epoch [3/200] Batch 0/6                       Loss D: 0.38676339387893677, loss G: 44.62611770629883
Epoch [4/200] Batch 0/6                       Loss D: 0.31247466802597046, loss G: 40.675174713134766
Epoch [5/200] Batch 0/6                       Loss D: 0.2677774429321289, loss G: 38.41364669799805
Epoch [6/200] Batch 0/6                       Loss D: 0.23058564960956573, loss G: 35.30905532836914
Epoch [7/200] Batch 0/6                       Loss D: 0.23361793160438538, loss G: 34.387123107910156
Epoch [8/200] Batch 0/6                       Loss D: 0.1881713569164276, loss G: 31.21179962158203
Epoch [9/200] Batch 0/6                       Loss D: 0.18105852603912354, loss G: 28.9760589